In [1]:
#import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable

from itertools import count
from collections import namedtuple


In [2]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [223]:
from scipy.spatial.distance import euclidean
import numpy as np
from numpy.random import randint
from itertools import permutations
from random import shuffle

import matplotlib.animation as animation
from IPython.display import HTML

"""
Actions:
    -Draw vertex
    -Measure distance
    -Measure angle
"""
# DEFINE ENVIRONMENT
class GeometryEnvironment():
    def __init__(self):
        self.reset()

    def place_vertex(self, bead_index, vertex_index):
        self.bead_set[bead_index] = vertex_index

    def measure_distance(self, i, j):
        u = self.vertex_set[i]
        v = self.vertex_set[j]
        return euclidean(u, v)
    
    def reset(self, get_complex_reward = True):
        self.get_complex_reward = get_complex_reward
        self.bead_set = np.array([random.randint(0, 3) for i in range(2)])
        self.defining_state = (randint(low=1, high=100), randint(low=1, high=100))
        x = self.defining_state
        self.vertex_set = np.array([(0,0), (x[0],0), (0,x[1]), x])
        self.answer = 0
        self.correct_answer = np.sqrt(self.defining_state[0]**2 + self.defining_state[1]**2)
    
    def basic_reward(self):
        return -(np.abs(self.answer - self.correct_answer)**2)
    
    def complex_reward(self):
        return 10.0 if self.answer == self.correct_answer else 0.0
    
    def step(self, action_index):
        # if distance should be measured
        #print("beads at start of step", self.bead_set)
        if (action_index == 8):
            self.answer = self.measure_distance(self.bead_set[0], self.bead_set[1])
        else:
            vertex_index = action_index % 4
            if (action_index < 5):
                # move first bead
                self.place_vertex(0, vertex_index)
            else:
                self.place_vertex(1, vertex_index)
        done = self.answer == self.correct_answer
        reward = self.complex_reward() if self.get_complex_reward else self.basic_reward()
        return self.get_state(), reward, done, -1
    
    def get_state(self):
        a = np.array(self.defining_state)
        b = np.array(self.vertex_set[(self.bead_set[0])])
        c = np.array(self.vertex_set[self.bead_set[1]])
        nump = np.concatenate((a, b, c))
        t_state = torch.tensor(nump, dtype=torch.long).float()
        t_state.requires_grad = True
        t_state.flatten()
        return t_state
    
    def render(self):
        #print("Answer: ", self.answer)
        vertices = np.array([self.vertex_set[(self.bead_set[0])], self.vertex_set[(self.bead_set[1])]])
        #print(vertices)
        x, y = vertices.T
        plt.scatter(x, y)
        plt.show()
    
    def set_state(self, x, y):
        self.defining_state = (int(x), int(y))
        x = self.defining_state
        self.vertex_set = np.array([(0,0), (x[0],0), (0,x[1]), x])
        self.answer = 0
        self.correct_answer = np.sqrt(self.defining_state[0]**2 + self.defining_state[1]**2)

    
    def sample_action_space(self):
        random_decision = random.randint(0, 2)
        if random_decision == 0:
            return torch.tensor([[8]], device=device, dtype=torch.long)    
        return torch.tensor([[random.randrange(9)]], device=device, dtype=torch.long)
    
    def get_distances(self):
        # self.answer,
        return self.measure_distance(self.bead_set[0], self.bead_set[1]), self.correct_answer


In [224]:
# REPLAY MEMORY DEFINITION
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Problems:

-Need to provide float args to vertex position (x and y)

-Need to provide index argument to vertex position

-Need to provide index arguments to distance measurement



In [225]:
# DEFINE THE NETWORK
# TODO: replace CNN with linear learner

def print_with_extreme_prejudice(num):
    print("Fuck you you fucking piece of shit", num)

class Actor(nn.Module):
    def __init__(self, input_dimension, output_dimension):
        super(Actor, self).__init__()

        self.fc1 = nn.Linear(input_dimension, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.head = nn.Linear(32, output_dimension)

    # Called with either one element to determine next action, or a batch
    # during optimization
    # Returns a distribution over the possible actions tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.float()
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        self.out = self.head(x.view(1, x.size(0)))[0]
        return  self.out # self.head(x.view(x.size(0), -1))

class Critic(nn.Module):
    def __init__(self, input_dimension, output_dimension):
        super(Critic, self).__init__()

        """
        self.fc1 = nn.Linear(input_dimension, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.head = nn.Linear(32, output_dimension)
        
        """
        self.state_input = torch.randn((1, 6), requires_grad=True)

        self.state1 = nn.Linear(6, 32)  # 6*6 from image dimension
        self.state2 = nn.Linear(32, 64)

        self.action_input = torch.randn((1, 1), requires_grad=True)
        self.action_input.retain_grad()

        self.merged1 = nn.Linear(65, 32)

        self.head = nn.Linear(32, output_dimension)
        self.out = torch.randn((1), requires_grad=True)

    # Called with either one element to determine next action, or a batch
    # during optimization
    # Returns a distribution over the possible actions tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        self.action_input.register_hook(print_with_extreme_prejudice)
        x = x.float()
        #print(x.shape)
        self.state_input = x[:, 0:6].clone()
        x_state = F.relu(self.state1(self.state_input))
        x_state = F.relu(self.state2(x_state))

        self.action_input = x[:, 6].clone().unsqueeze(0)
        x_action = self.action_input

        print(x_state)
        print(x_action)
        x = torch.cat((x_state, x_action), 1)
        x = F.relu(self.merged1(x))
        self.out = self.head(x.view(x.size(0), -1)) # self.head(x.view(x.size(0), -1))
        return self.out

In [229]:
import random
from collections import deque
def clone_tensors(item_list):
    new_items = []
    for item in item_list:
        new_items.append(item.clone() if torch.is_tensor(item) else item)
    return new_items

class ActorCritic:
    def __init__(self, env):
        self.env  = env

        self.action_dimension = 9
        self.environment_dimension = 6     
        
        self.learning_rate = 0.01 #0.001
        self.epsilon = 1.0
        self.epsilon_decay = 50 #.995
        self.gamma = 0.9 #.95
        self.tau   = .125
        self.memory = deque(maxlen=2000)

        # ===================================================================== #
        #                               Actor Model                             #
        # Chain rule: find the gradient of changing the actor network params in #
        # getting closest to the final value network predictions, i.e. de/dA    #
        # Calculate de/dA as = de/dC * dC/dA, where e is error, C critic, A act #
        # ===================================================================== #

        self.memory = deque(maxlen=2000)

        self.actor_model = Actor(self.environment_dimension, self.action_dimension).float()
        self.target_actor_model = Actor(self.environment_dimension, self.action_dimension).float()

        self.target_actor_model.load_state_dict(self.actor_model.state_dict())
        self.target_actor_model.eval()

        #self.actor_optimizer = optim.Adam(self.actor_model.parameters())
        self.actor_optimizer = optim.RMSprop(self.actor_model.parameters())
        
        #print(self.actor_model.state_dict())
        """
        self.actor_critic_grad = tf.placeholder(tf.float32, 
            [None, self.env.action_space.shape[0]]) # where we will feed de/dC (from critic)
        
        actor_model_weights = self.actor_model.trainable_weights
        self.actor_grads = tf.gradients(self.actor_model.output, 
            actor_model_weights, -self.actor_critic_grad) # dC/dA (from actor)
        
        grads = zip(self.actor_grads, actor_model_weights)

        
        self.optimize = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(grads)
         optimizer = optim.Adam(policy_net.parameters())
         """

        # ===================================================================== #
        #                              Critic Model                             #
        # ===================================================================== #        

        self.critic_model = Critic(self.action_dimension + self.environment_dimension, 1).float()
        self.target_critic_model = Critic(self.action_dimension + self.environment_dimension, 1).float()

        self.target_critic_model.load_state_dict(self.critic_model.state_dict())
        self.target_critic_model.eval()

        #self.critic_grads =  torch.autograd.grad(self.critic_model.head, self.critic_model.action_input)

        #self.critic_optimizer = optim.Adam(self.critic_model.parameters())
        self.critic_optimizer = optim.RMSprop(self.critic_model.parameters())

        """
        self.critic_grads = tf.gradients(self.critic_model.output, self.critic_action_input) # where we calcaulte de/        dC for feeding above
        optimizer = optim.Adam(policy_net.parameters())
        """
        
    # ===================================================================== #
    #                              Training                                 #
    # ===================================================================== #    

    def remember(self, cur_state, action, reward, new_state, done):
        self.memory.append([cur_state, action, reward, new_state, done])
    
    def _train_actor(self, samples):
        for real_sample in samples:
            sample = clone_tensors(real_sample)
            # Want to get critic grad, but with respect to the predicted action as the action input
            cur_state, action, reward, new_state, _ = sample

            # get action input
            #self.actor_optimizer.zero_grad()
            predicted_action = self.actor_model(cur_state)
            predicted_action = predicted_action.max(0)[1].view(1, 1).float()
            
            # get critic predictions
            critic_input = torch.cat((cur_state, predicted_action.flatten()))
            expected_rewards = self.critic_model(critic_input.unsqueeze(0))

            prediction_critic_grad = torch.autograd.grad(self.critic_model.out, self.critic_model.action_input, grad_outputs=expected_rewards)[0].clone()

            actor_model_weights = self.actor_model.parameters()
            
            # Compute actor gradients

            self.actor_grads = torch.autograd.grad(self.actor_model.out, actor_model_weights, grad_outputs=-prediction_critic_grad)

            # Set up relation between actor grads and weights
            zipperino = zip(list(self.actor_model.parameters()), self.actor_grads)
            
            for param, gradient in zipperino:
                param.grad = gradient.clone()
            
            # Step optimizer
            #self.actor_optimizer.step()
            
    def _train_critic(self, samples):
        for real_sample in samples:
            sample = clone_tensors(real_sample)
            cur_state, action, reward, new_state, done = sample
            if not done:
                target_action = self.target_actor_model(new_state)
                # get actual value of target action
                target_action = target_action.max(0)[1].view(1, 1).float()
                # include predicted future reward in reward
                #print(target_action)
                critic_input = torch.cat((new_state.float(), target_action.flatten()), 0)
                print(critic_input)
                future_reward = self.target_critic_model(critic_input.unsqueeze(0))[0]
                reward += self.gamma * future_reward
            # train model
            optim_critic_input = torch.cat((cur_state.float(), action.flatten().float()), axis=0)

            self.optimize_critic(optim_critic_input, torch.tensor(reward).view(1,1))
    
    def optimize_critic(self, input_state, reward):
        # optimize critic on a batch
        print(input_state)

        # get critic predictions
        expected_rewards = self.critic_model(input_state.unsqueeze(0))

        # Optimize the model
        self.actor_optimizer.zero_grad()
        self.critic_optimizer.zero_grad()

        # Compute loss
        loss = F.smooth_l1_loss(expected_rewards, reward.unsqueeze(1))
        #print(loss)


        #self.critic_model.action_input.retain_grad()
        #loss = loss.double()
        loss.backward(retain_graph=True)
        #print("Fuck: ", self.critic_model.action_input.grad)
        self.critic_grad = torch.autograd.grad(self.critic_model.out, self.critic_model.action_input)[0]

        for param in self.critic_model.parameters():
            param.grad.data.clamp_(-1, 1)

        self.critic_optimizer.step()

        
    def train(self):
        batch_size = 32
        if len(self.memory) < batch_size:
            return

        rewards = []
        samples = random.sample(self.memory, batch_size)
        self._train_critic(samples)
        self._train_actor(samples)

    # ========================================================================= #
    #                         Target Model Updating                             #
    # ========================================================================= #

    def _update_actor_target(self):
        self.target_actor_model.load_state_dict(self.actor_model.state_dict())

    def _update_critic_target(self):
        self.target_critic_model.load_state_dict(self.critic_model.state_dict())

    def update_target(self):
        self._update_actor_target()
        self._update_critic_target()

    # ========================================================================= #
    #                              Model Predictions                            #
    # ========================================================================= #

    def act(self, cur_state):
        self.epsilon *= self.epsilon_decay

        if np.random.random() < self.epsilon:
            return self.env.sample_action_space()
        
        return self.actor_model(cur_state)
    
    def test(self):
        test_states = []
        test_actions = []
        test_distances = []

        self.env.reset()
        state = self.env.get_state()
        for t in count():
            print(t)
            #print(state)
            # Select and perform an action
            action = self.act(state)
            __, reward, done, __ = self.env.step(action)
            reward = torch.tensor([reward], device=device)

            pred_dist, actual_dist = self.env.get_distances()

            test_states.append(state)
            test_actions.append(action[0])
            test_distances.append([pred_dist, actual_dist])

            # Observe new state
            if not done:
                next_state = self.env.get_state()
            else:
                next_state = None


            # Move to the next state
            state = next_state

            if done or t > 999:
                print(t)
                break
            
        measurements = np.array(test_actions) < 30
        print(measurements)
        #=========================================
        # Animate Images

        fig = plt.figure(figsize=(10, 10))
        ims = []
        for i in range(len(test_states)):
            actual_value = np.array(test_states[i].detach())[0:2]
            distances = test_distances[i]
            fig_title = "a = " + str(actual_value[0]) + ", b = " + str(actual_value[1]) + " test distance: " + str(distances[0]) + " actual distance: " + str(distances[1])
            x = np.array(test_states[i].detach())[2:5:2]
            y = np.array(test_states[i].detach())[3:6:2]
            print("x: ", x)
            print("y: ", y)
            if (measurements[i]):
                print("measurement")
                """
                ims.append([plt.scatter(x, y, animated=True)])
                im, = plt.plot(x, y, 'ro-', animated=True)
                """
                plt.plot(x, y, 'ro-', animated=True)
                plt.title(fig_title)
                plt.show()
            else:
                plt.title(fig_title)
                plt.scatter(x, y, animated=True)
                plt.show()
                #im = plt.scatter(x, y, animated=True)
            #ims.append([im])

        ani = animation.ArtistAnimation(fig, ims, interval=300, blit=True, repeat_delay=1000)
        #plt.close()

        # Show the animation
        HTML(ani.to_jshtml())

    def test_count(self):
        self.env.reset()
        state = self.env.get_state()
        for t in count():
            #print(state)
            # Select and perform an action
            action = self.act(state)
            __, reward, done, __ = self.env.step(action)
            reward = torch.tensor([reward], device=device)

            # Observe new state
            if not done:
                next_state = self.env.get_state()
            else:
                next_state = None


            # Move to the next state
            state = next_state

            if done or t > 999:
                print("Required ", t, " steps")
                break

    


In [230]:
torch.autograd.set_detect_anomaly(False)
def main():
    env = GeometryEnvironment()
    actor_critic = ActorCritic(env)

    num_trials = 2000
    trial_len  = 999

    cur_state = env.reset()
    action = env.sample_action_space()
    count = 0
    while count < num_trials:
        count += 1
        cur_state = env.get_state()

        action = actor_critic.act(cur_state)
        #action = action[0] #action.reshape((1, env.action_space.shape[0]))

        new_state, reward, done, _ = env.step(action)
        #new_state = new_state.reshape((1, env.observation_space.shape[0]))

        actor_critic.remember(cur_state, action, reward, new_state, done)
        actor_critic.train()

        cur_state = new_state
        if (count % 10 == 0):
            print("Episode: ", count)
            actor_critic.test_count()
            actor_critic.update_target()
    
    actor_critic.test()

In [231]:
main()

Episode:  10
Required  20  steps
Episode:  20
Required  2  steps
Episode:  30
Required  7  steps
tensor([23., 64.,  0.,  0., 23.,  0.,  1.], grad_fn=<CatBackward>)
tensor([[15.0448,  1.1981,  4.5461,  1.9848,  6.2570,  0.0000,  0.0000, 10.7930,
          0.0000,  0.0000,  7.3734,  0.0000,  0.0000,  0.0000, 20.9615,  0.0000,
         10.8520,  0.0000,  0.0000,  0.0000,  4.7871, 10.0066, 11.5203,  0.6246,
          0.0866,  0.6717,  0.0000,  9.2722,  0.0000,  0.0000,  2.4353,  5.4947,
          0.0000,  8.9224, 12.9772,  0.0000,  0.0382,  0.4007,  5.0107,  0.0000,
          0.0000,  0.4666,  0.7026,  0.0000,  0.3458,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  4.1490,  0.0000,  0.0000, 12.8572,  0.0000,
          0.0000,  0.0000,  6.1417, 11.2532,  0.0000, 13.7491,  7.1651,  0.0000]],
       grad_fn=<ReluBackward0>)
tensor([[1.]], grad_fn=<UnsqueezeBackward0>)
tensor([23., 64.,  0.,  0., 23.,  0.,  8.], grad_fn=<CatBackward>)
tensor([[15.0448,  1.1981,  4.5461,  1.984

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([1, 1]) and output[0] has a shape of torch.Size([9]).